# Imports

In [1]:
import os, shutil
import time
import sqlite3
import csv
import json
import logging
from psutil import virtual_memory
import pandas as pd
import requests
from pprint import pprint
from elasticsearch_dsl import Index, connections
from elasticsearch import helpers
import logging

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
def mem():
    print(f'used memory : {round(virtual_memory()[3]/(1024*1024*1024)*10)/10}Go')

In [4]:
def stats(): 
    print("--- %s seconds ---" % (time.time() - start_time))
    mem()

In [5]:
def global_stats(): 
    print("--- %s seconds ---" % (time.time() - global_start_time))
    mem()

In [6]:
def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist

def unique_string(a):
    return ' '.join(unique_list(a.strip().split(','))).strip()

In [7]:
def get_string(string):
    if string is None:
        return ""
    return string

In [8]:
mem()

used memory : 13.6Go


In [9]:
global_start_time = time.time()

In [10]:
print('RAM memory % used:', virtual_memory()[2])

RAM memory % used: 24.5


# Path set-up

In [11]:
if "DATA_DIR" not in locals():
    DATA_DIR = "./data/"
else:
    print(DATA_DIR)

if os.path.exists(DATA_DIR) and os.path.isdir(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(os.path.dirname(DATA_DIR), exist_ok=True)

In [12]:
if "OUTPUT_DATA_FOLDER" not in locals():
    OUTPUT_DATA_FOLDER = "./output/"
else:
    print(OUTPUT_DATA_FOLDER)

if os.path.exists(OUTPUT_DATA_FOLDER) and os.path.isdir(OUTPUT_DATA_FOLDER):
    shutil.rmtree(OUTPUT_DATA_FOLDER)
os.makedirs(os.path.dirname(OUTPUT_DATA_FOLDER), exist_ok=True)

# Prepare Data Enrichment 

## Nom complet

In [13]:
def create_nom_complet(nature_juridique_unite_legale=None, nom=None, nom_usage=None, nom_raison_sociale=None, sigle=None, prenom=None):    
    is_auto_entrepreneur = nature_juridique_unite_legale == "1000"
    
    if is_auto_entrepreneur:
        formatted_nom_usage = " " + nom_usage if nom_usage else ""
        formatted_sigle = sigle if sigle else ""
        
        if (prenom is None and nom is None):
            return None
        if nom_usage is None:
            return f"{prenom} {nom} ({formatted_sigle})".lower()
        else:
            return f"{prenom} {formatted_nom_usage}({nom}, {formatted_sigle})".lower()
            
    else:
        if nom_raison_sociale is None and sigle is None:
            return None
        else:
            formatted_sigle = f" ({sigle})" if sigle else ""
            return f"{nom_raison_sociale}{formatted_sigle}".lower()

## Entrepreneur Individuel

In [14]:
def create_entrepreneur_individuel(nature_juridique_unite_legale):
    if nature_juridique_unite_legale in ["1", "10", "1000"]:
        return 'true'
    else:
        return 'false'

## Section activité principale

In [15]:
sections_NAF = {
"01":"A","02":"A","03":"A","05":"B","06":"B","07":"B","08":"B","09":"B","10":"C","11":"C","12":"C","13":"C","14":"C",
 "15":"C","16":"C","17":"C","18":"C","19":"C","20":"C","21":"C","22":"C","23":"C","24":"C","25":"C","26":"C","27":"C",
 "28":"C","29":"C","30":"C","31":"C","32":"C","33":"C","35":"D","36":"E","37":"E","38":"E","39":"E","41":"F","42":"F",
 "43":"F","45":"G","46":"G","47":"G","49":"H","50":"H","51":"H","52":"H","53":"H","55":"I","56":"I","58":"J","59":"J",
 "60":"J","61":"J","62":"J","63":"J","64":"K","65":"K","66":"K","68":"L","69":"M","70":"M","71":"M","72":"M","73":"M",
 "74":"M","75":"M","77":"N","78":"N","79":"N","80":"N","81":"N","82":"N","84":"O","85":"P","86":"Q","87":"Q","88":"Q",
 "90":"R","91":"R","92":"R","93":"R","94":"S","95":"S","96":"S","97":"T","98":"T","99":"U"
}

In [16]:
def create_section(activite_principale_unite_legale):
    if activite_principale_unite_legale is not None:
        code_naf = activite_principale_unite_legale[:2]
        section_activite_principale = sections_NAF[code_naf] if code_naf in sections_NAF else None
        return section_activite_principale
    else:
        return None

## Adresse complète

In [17]:
def create_adresse_complete(complement_adresse, numero_voie, indice_repetition, type_voie, libelle_voie, libelle_commune, libelle_cedex, distribution_speciale, commune, cedex, libelle_commune_etranger, libelle_pays_etranger):    
    col_list = [complement_adresse, numero_voie, indice_repetition, type_voie, libelle_voie, distribution_speciale]
    adresse = ""
    for column in col_list:
        if(column !=''):
            adresse = adresse + " " + column if column is not None else ""
    if cedex == "":
        if commune == "":
            adresse =  adresse
        else:
            adresse = adresse + " " + commune + " " + libelle_commune
    else:
        adresse = adresse + " " + cedex + " " + libelle_cedex
    etranger_list = [libelle_commune_etranger, libelle_pays_etranger]
    for column in etranger_list:
        if(column != ""):
            adresse = adresse + " " + column if column != "" else ""
    return adresse.strip()

## Département

In [18]:
def create_departement(commune):
    departement = str(commune)[:3] if str(commune)[:2]=="97" else (None if commune is None else str(commune)[:2])
    return departement

## Coordonnées

In [19]:
def create_coordonnees(longitude, latitude):
    coordonnees = None if (longitude is None) or (latitude is None) else f"{latitude},{longitude}"
    return coordonnees

## Liste enseigne

In [20]:
def create_list_enseignes(enseigne_1=None, enseigne_2=None, enseigne_3=None, nom_commercial=None):
    return list(filter(None,set([enseigne_1, enseigne_2, enseigne_3, nom_commercial])))

# SQLITE


## Set-up

In [21]:
if os.path.exists(DATA_DIR+'sirene.db'):
    os.remove(DATA_DIR+'sirene.db')

In [22]:
connection = sqlite3.connect(DATA_DIR+'sirene.db')

In [23]:
cursor = connection.cursor()

## Unité Légale

In [24]:
cursor.execute(f'''DROP TABLE IF EXISTS unite_legale''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS unite_legale
    (
        siren,
        date_creation_unite_legale,
        sigle,
        prenom,
        identifiant_association_unite_legale,
        tranche_effectif_salarie_unite_legale,
        date_mise_a_jour_unite_legale,
        categorie_entreprise,
        etat_administratif_unite_legale,
        nom,
        nom_usage,
        nom_raison_sociale,
        nature_juridique_unite_legale,
        activite_principale_unite_legale,
        economie_sociale_solidaire_unite_legale
    )
''')

In [25]:
cursor.execute('''
                CREATE UNIQUE INDEX index_siren
                ON unite_legale (siren);
                ''')

In [26]:
connection.commit()

In [27]:
start_time = time.time()
url = 'https://files.data.gouv.fr/insee-sirene/StockUniteLegale_utf8.zip'
r = requests.get(url, allow_redirects=True)
open(DATA_DIR+'StockUniteLegale_utf8.zip', 'wb').write(r.content)
shutil.unpack_archive(DATA_DIR+'StockUniteLegale_utf8.zip', DATA_DIR)
stats()

--- 17.79559326171875 seconds ---
used memory : 14.3Go


In [28]:
df_iterator = pd.read_csv(
    DATA_DIR+'StockUniteLegale_utf8.csv', 
    chunksize=100000,
    dtype=str)

In [29]:
start_time = time.time()

for i, df_unite_legale in enumerate(df_iterator):
    df_unite_legale = df_unite_legale[[
            "siren",
            "dateCreationUniteLegale",
            "sigleUniteLegale",
            "prenom1UniteLegale",
            "identifiantAssociationUniteLegale",
            "trancheEffectifsUniteLegale",
            "dateDernierTraitementUniteLegale",
            "categorieEntreprise",
            "etatAdministratifUniteLegale",
            "nomUniteLegale",
            "nomUsageUniteLegale",
            "denominationUniteLegale",
            "categorieJuridiqueUniteLegale",
            "activitePrincipaleUniteLegale",
            "economieSocialeSolidaireUniteLegale",
        ]]
    # Rename columns
    df_unite_legale = df_unite_legale.rename(
        columns={
            "dateCreationUniteLegale": "date_creation_unite_legale",
            "sigleUniteLegale": "sigle",
            "prenom1UniteLegale": "prenom",
            "trancheEffectifsUniteLegale": "tranche_effectif_salarie_unite_legale",
            "dateDernierTraitementUniteLegale": "date_mise_a_jour_unite_legale",
            "categorieEntreprise": "categorie_entreprise",
            "etatAdministratifUniteLegale":"etat_administratif_unite_legale",
            "nomUniteLegale": "nom",
            "nomUsageUniteLegale": "nom_usage",
            "denominationUniteLegale": "nom_raison_sociale",
            "categorieJuridiqueUniteLegale": "nature_juridique_unite_legale",
            "activitePrincipaleUniteLegale": "activite_principale_unite_legale",
            "economieSocialeSolidaireUniteLegale":"economie_sociale_solidaire_unite_legale",
            "identifiantAssociationUniteLegale":"identifiant_association_unite_legale",
        }
    )    
    df_unite_legale.to_sql("unite_legale", connection, if_exists='append', index=False)

stats()
del df_unite_legale

--- 233.75794911384583 seconds ---
used memory : 14.4Go


In [30]:
connection.commit()

In [31]:
for row in cursor.execute("select count() from unite_legale"):
    print(row)

(23333628,)


In [32]:
for item in cursor.execute('PRAGMA table_info(unite_legale)'):
    print (item)

(0, 'siren', '', 0, None, 0)
(1, 'date_creation_unite_legale', '', 0, None, 0)
(2, 'sigle', '', 0, None, 0)
(3, 'prenom', '', 0, None, 0)
(4, 'identifiant_association_unite_legale', '', 0, None, 0)
(5, 'tranche_effectif_salarie_unite_legale', '', 0, None, 0)
(6, 'date_mise_a_jour_unite_legale', '', 0, None, 0)
(7, 'categorie_entreprise', '', 0, None, 0)
(8, 'etat_administratif_unite_legale', '', 0, None, 0)
(9, 'nom', '', 0, None, 0)
(10, 'nom_usage', '', 0, None, 0)
(11, 'nom_raison_sociale', '', 0, None, 0)
(12, 'nature_juridique_unite_legale', '', 0, None, 0)
(13, 'activite_principale_unite_legale', '', 0, None, 0)
(14, 'economie_sociale_solidaire_unite_legale', '', 0, None, 0)


In [33]:
for row in cursor.execute("SELECT * FROM unite_legale WHERE siren='965706690' LIMIT 10;"):
    print(row)

('965706690', None, None, 'ANNIE', None, None, None, None, 'C', 'MONNIER', 'BILLAUD', None, '1000', '00.97', None)


In [34]:
cursor.execute('SELECT * FROM unite_legale LIMIT 1')
names = [description[0] for description in cursor.description]
rows = cursor.fetchall()
for row in rows:
    for name,val in zip(names,row):
        print(f"{name}: {val}")

siren: 000325175
date_creation_unite_legale: 2000-09-26
sigle: None
prenom: THIERRY
identifiant_association_unite_legale: None
tranche_effectif_salarie_unite_legale: None
date_mise_a_jour_unite_legale: 2019-12-13T13:21:28
categorie_entreprise: PME
etat_administratif_unite_legale: A
nom: JANOYER
nom_usage: None
nom_raison_sociale: None
nature_juridique_unite_legale: 1000
activite_principale_unite_legale: 32.12Z
economie_sociale_solidaire_unite_legale: None


## Établissements

In [35]:
# Create list of departement zip codes
all_deps = [
    *"-0".join(list(str(x) for x in range(0, 10))).split("-")[1:],
    *list(str(x) for x in range(10, 20)),
    *["2A", "2B"],
    *list(str(x) for x in range(21, 96)),
    *"-7510".join(list(str(x) for x in range(0, 10))).split("-")[1:],
    *"-751".join(list(str(x) for x in range(10, 21))).split("-")[1:],
    *["971", "972", "973", "974", "976"],
    *[""],
]
# Remove Paris zip code
all_deps.remove("75")

In [36]:
cursor.execute(f'''DROP TABLE IF EXISTS siret''')
cursor.execute(f'''CREATE TABLE IF NOT EXISTS siret
        (
        id INTEGER NOT NULL PRIMARY KEY,
        siren,
        siret,
        date_creation,
        tranche_effectif_salarie,
        activite_principale_registre_metier,
        is_siege,
        numero_voie,
        type_voie,
        libelle_voie,
        code_postal,
        libelle_cedex,
        libelle_commune,
        commune,
        complement_adresse,
        complement_adresse_2,
        numero_voie_2,
        indice_repetition_2,
        type_voie_2,
        libelle_voie_2,
        commune_2,
        libelle_commune_2,
        cedex_2,
        libelle_cedex_2,
        cedex,
        date_debut_activite,
        distribution_speciale,
        distribution_speciale_2,
        etat_administratif_etablissement,
        enseigne_1,
        enseigne_2,
        enseigne_3,
        activite_principale,
        indice_repetition,
        nom_commercial,
        libelle_commune_etranger,
        code_pays_etranger,
        libelle_pays_etranger,
        libelle_commune_etranger_2,
        code_pays_etranger_2,
        libelle_pays_etranger_2,
        longitude,
        latitude,
        geo_adresse,
        geo_id)
        ''')

In [37]:
%%time
# Upload geo data by departement
for dep in all_deps:
    start_time = time.time()
    url = "https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_" + dep + ".csv.gz"
    print(url)
    df_dep = pd.read_csv(
        url,
        compression="gzip",
        dtype=str,
        usecols=[
            "siren",
            "siret",
            "dateCreationEtablissement",
            "trancheEffectifsEtablissement",
            "activitePrincipaleRegistreMetiersEtablissement",
            "etablissementSiege",
            "numeroVoieEtablissement",
            "libelleVoieEtablissement",
            "codePostalEtablissement",
            "libelleCommuneEtablissement",
            "libelleCedexEtablissement",
            "typeVoieEtablissement",
            "codeCommuneEtablissement",
            "codeCedexEtablissement",
            "complementAdresseEtablissement",
            "distributionSpecialeEtablissement",
            "complementAdresse2Etablissement",
            "indiceRepetition2Etablissement",
            "libelleCedex2Etablissement",
            "codeCedex2Etablissement",
            "numeroVoie2Etablissement",
            "typeVoie2Etablissement",
            "libelleVoie2Etablissement",
            "codeCommune2Etablissement",
            "libelleCommune2Etablissement",
            "distributionSpeciale2Etablissement",
            "dateDebut",
            "etatAdministratifEtablissement",
            "enseigne1Etablissement",
            "enseigne1Etablissement",
            "enseigne2Etablissement",
            "enseigne3Etablissement",
            "denominationUsuelleEtablissement",
            "activitePrincipaleEtablissement",
            "geo_adresse",
            "geo_id",
            "longitude",
            "latitude",
            "indiceRepetitionEtablissement",
            "libelleCommuneEtrangerEtablissement",
            "codePaysEtrangerEtablissement",
            "libellePaysEtrangerEtablissement",
            "libelleCommuneEtranger2Etablissement",
            "codePaysEtranger2Etablissement",
            "libellePaysEtranger2Etablissement",
        ],
    )
    df_dep = df_dep.rename(
        columns={
            "dateCreationEtablissement": "date_creation",
            "trancheEffectifsEtablissement": "tranche_effectif_salarie",
            "activitePrincipaleRegistreMetiersEtablissement": "activite_principale_registre_metier",
            "etablissementSiege": "is_siege",
            "numeroVoieEtablissement": "numero_voie",
            "typeVoieEtablissement": "type_voie",
            "libelleVoieEtablissement": "libelle_voie",
            "codePostalEtablissement": "code_postal",
            "libelleCedexEtablissement": "libelle_cedex",
            "libelleCommuneEtablissement": "libelle_commune",
            "codeCommuneEtablissement": "commune",
            "complementAdresseEtablissement": "complement_adresse",
            "complementAdresse2Etablissement": "complement_adresse_2",
            "numeroVoie2Etablissement": "numero_voie_2",
            "indiceRepetition2Etablissement": "indice_repetition_2",
            "typeVoie2Etablissement": "type_voie_2",
            "libelleVoie2Etablissement": "libelle_voie_2",
            "codeCommune2Etablissement": "commune_2",
            "libelleCommune2Etablissement": "libelle_commune_2",
            "codeCedex2Etablissement": "cedex_2",
            "libelleCedex2Etablissement": "libelle_cedex_2",
            "codeCedexEtablissement": "cedex",
            "dateDebut": "date_debut_activite",
            "distributionSpecialeEtablissement": "distribution_speciale",
            "distributionSpeciale2Etablissement": "distribution_speciale_2",
            "etatAdministratifEtablissement": "etat_administratif_etablissement",
            "enseigne1Etablissement": "enseigne_1",
            "enseigne2Etablissement": "enseigne_2",
            "enseigne3Etablissement": "enseigne_3",
            "activitePrincipaleEtablissement": "activite_principale",
            "indiceRepetitionEtablissement": "indice_repetition",
            "denominationUsuelleEtablissement": "nom_commercial",
            "libelleCommuneEtrangerEtablissement": "libelle_commune_etranger",
            "codePaysEtrangerEtablissement": "code_pays_etranger",
            "libellePaysEtrangerEtablissement": "libelle_pays_etranger",
            "libelleCommuneEtranger2Etablissement": "libelle_commune_etranger_2",
            "codePaysEtranger2Etablissement": "code_pays_etranger_2",
            "libellePaysEtranger2Etablissement": "libelle_pays_etranger_2",
        }
    )
    stats()
    start_time = time.time()
    df_dep.to_sql("siret", connection, if_exists='append', index=False)
    connection.commit()
    stats()
del df_dep

https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_01.csv.gz
--- 1.9009130001068115 seconds ---
used memory : 14.7Go
--- 4.058767795562744 seconds ---
used memory : 14.5Go
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_02.csv.gz
--- 1.450409173965454 seconds ---
used memory : 14.6Go
--- 3.052818775177002 seconds ---
used memory : 14.6Go
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_03.csv.gz
--- 1.1834681034088135 seconds ---
used memory : 14.6Go
--- 2.6561670303344727 seconds ---
used memory : 14.6Go
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_04.csv.gz
--- 0.8032910823822021 seconds ---
used memory : 14.5Go
--- 1.7088181972503662 seconds ---
used memory : 14.5Go
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_05.csv.gz
--- 0.8387877941131592 seconds ---
used memory : 14.5Go
--- 1.7336652278900146 seconds ---
used memory : 14.5Go
https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_06.csv.gz
--- 6.608724355697632 seconds ---
used me

In [38]:
for row in cursor.execute("select count() from siret"):
    print(row)

(32849246,)


In [39]:
for row in cursor.execute('SELECT * FROM siret LIMIT 2;'):
    print(row)

(1, '005520135', '00552013500038', '2007-04-20', None, None, 'true', '70', 'RUE', 'DE LAUSANNE', '01220', None, 'DIVONNE-LES-BAINS', '01143', None, None, None, None, None, None, None, None, None, None, None, '2007-11-19', None, None, 'F', None, None, None, '17.1P', None, None, None, None, None, None, None, None, '6.143376', '46.357963', '70 Rue de Lausanne 01220 Divonne-les-Bains', '01143_0490_00070')
(2, '015550189', '01555018900094', '1993-03-17', None, None, 'false', None, 'LD', 'CHAMPAGNE', '01540', None, 'VONNAS', '01457', None, None, None, None, None, None, None, None, None, None, None, '1993-12-25', None, None, 'F', None, None, None, '45.2P', None, None, None, None, None, None, None, None, '4.99279', '46.219693', 'Impasse du Champ de Tir 01540 Vonnas', '01457_0009')


In [40]:
cursor.execute('SELECT * FROM siret LIMIT 1')
names = [description[0] for description in cursor.description]
rows = cursor.fetchall()
for row in rows:
    for name,val in zip(names,row):
        print(f"{name}: {val}")

id: 1
siren: 005520135
siret: 00552013500038
date_creation: 2007-04-20
tranche_effectif_salarie: None
activite_principale_registre_metier: None
is_siege: true
numero_voie: 70
type_voie: RUE
libelle_voie: DE LAUSANNE
code_postal: 01220
libelle_cedex: None
libelle_commune: DIVONNE-LES-BAINS
commune: 01143
complement_adresse: None
complement_adresse_2: None
numero_voie_2: None
indice_repetition_2: None
type_voie_2: None
libelle_voie_2: None
commune_2: None
libelle_commune_2: None
cedex_2: None
libelle_cedex_2: None
cedex: None
date_debut_activite: 2007-11-19
distribution_speciale: None
distribution_speciale_2: None
etat_administratif_etablissement: F
enseigne_1: None
enseigne_2: None
enseigne_3: None
activite_principale: 17.1P
indice_repetition: None
nom_commercial: None
libelle_commune_etranger: None
code_pays_etranger: None
libelle_pays_etranger: None
libelle_commune_etranger_2: None
code_pays_etranger_2: None
libelle_pays_etranger_2: None
longitude: 6.143376
latitude: 46.357963
geo_adr

In [41]:
connection.commit()

### Nombre d'établissements

In [42]:
# create a count table
cursor.execute(f'''DROP TABLE IF EXISTS count_etab''')
cursor.execute('''CREATE TABLE count_etab (siren VARCHAR(10), count INTEGER)''')
# create index
cursor.execute('''
                CREATE UNIQUE INDEX index_count_siren
                ON count_etab (siren);
                ''')
connection.commit()

In [43]:
start_time = time.time()
# Add etab count
cursor.execute('''INSERT INTO count_etab (siren, count) SELECT siren, count(*) as count FROM siret GROUP BY siren;''')
stats()

--- 38.87159466743469 seconds ---
used memory : 14.6Go


In [44]:
for row in cursor.execute("select count() from count_etab"):
    print(row)

(23218129,)


In [45]:
for row in cursor.execute('SELECT * FROM count_etab LIMIT 5;'):
    print(row)

('000325175', 5)
('001807254', 2)
('005410220', 1)
('005410345', 1)
('005410394', 1)


### Nombre d'établissements ouverts

In [46]:
cursor.execute(f'''DROP TABLE IF EXISTS count_etab_ouvert''')
cursor.execute('''CREATE TABLE count_etab_ouvert (siren VARCHAR(10), count INTEGER)''')
cursor.execute('''
                CREATE UNIQUE INDEX index_count_ouvert_siren
                ON count_etab_ouvert (siren);
                ''')
connection.commit()

In [47]:
start_time = time.time()
# Add etab count
cursor.execute('''INSERT INTO count_etab_ouvert (siren, count) SELECT siren, count(*) as count FROM siret WHERE etat_administratif_etablissement = 'A' GROUP BY siren;''')
stats()

--- 21.621156215667725 seconds ---
used memory : 14.7Go


In [48]:
for row in cursor.execute("select count() from count_etab_ouvert"):
    print(row)

(11943871,)


In [49]:
for row in cursor.execute('SELECT * FROM count_etab_ouvert LIMIT 5;'):
    print(row)

('000325175', 1)
('005420021', 1)
('005420120', 3)
('005440110', 1)
('005440565', 1)


### Liste d'enseignes

In [50]:
# Add liste d'enseignes for each établissement
add_enseigne = f'''ALTER TABLE siret ADD COLUMN enseignes GENERATED ALWAYS AS
               (COALESCE(enseigne_1, '') || COALESCE(enseigne_2, ' ') || COALESCE(enseigne_3, ' ') || COALESCE(nom_commercial, ''))
               '''
cursor.execute(add_enseigne)

In [51]:
# Create enseignes table with grouped lists of enseignes per siren
cursor.execute(f'''DROP TABLE IF EXISTS enseignes''')
cursor.execute('''CREATE TABLE enseignes (siren VARCHAR(10), liste_enseignes)''')
cursor.execute('''
                CREATE UNIQUE INDEX index_liste_enseignes
                ON enseignes (siren);
                ''')
connection.commit()

In [52]:
start_time = time.time()
# Add liste enseignes
cursor.execute('''INSERT INTO enseignes (siren, liste_enseignes) SELECT siren, GROUP_CONCAT(enseignes, ',') as liste_enseignes FROM siret GROUP BY siren;''')
stats()

--- 57.21150088310242 seconds ---
used memory : 14.7Go


In [53]:
for row in cursor.execute('SELECT liste_enseignes FROM enseignes WHERE siren="345130488" LIMIT 2;'):
    print(row)

("  ,CARREFOUR CONTACT  ,CARREFOUR CONTACT  ,CARREFOUR CONTACT  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CARREFOUR CONTACT  ,CARREFOUR CONTACT  ,CARREFOUR EXPRESS  ,CARREFOUR CONTACT  ,  ,CARREFOUR EXPRESS  ,  ,  ,  ,CARREFOUR CITY  ,  ,  ,  ,CARREFOUR CONTACT  ,CARREFOUR CONTACT  ,8 A HUIT  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CARREFOUR CONTACT  ,CONTACT MARCHE  ,CARREFOUR CONTACT  ,  ,CARREFOUR CITY  ,SHOPI  ,CARREFOUR EXPRESS  ,  ,CARREFOUR CONTACT  ,CONTACT MARCHE  ,CONTACT MARCHE  ,  ,CARREFOUR CITY  ,  ,  ,  ,  ,8 A HUIT  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CARREFOUR CONTACT  ,CARREFOUR CONTACT  ,  ,  ,CARREFOUR CITY  ,CARREFOUR CONTACT  ,CARREFOUR CONTACT  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CONTACT MARCHE  ,CONTACT MARCHE  ,  ,CARREFOUR EXPRESS  ,  ,CARREFOUR EXPRESS  ,  ,CARREFOUR CITY  ,  ,CARREFOUR EXPRESS  ,CARREFOUR CONTACT  ,  ,CARREFOUR CITY  ,CARREFOUR CITY  ,CARREFOUR CITY  ,CARREFOUR CONTACT  ,

In [54]:
for row in cursor.execute('SELECT enseignes FROM siret WHERE siren="345130488" LIMIT 5;'):
    print(row)

('  ',)
('CARREFOUR CONTACT  ',)
('CARREFOUR CONTACT  ',)
('CARREFOUR CONTACT  ',)
('CONTACT MARCHE  ',)


### Liste adresses

In [55]:
# Create SQLite function
connection.create_function("add_adresse_complete", 12, create_adresse_complete)
# Add adresse_complete column for each établissement
cursor.execute('ALTER TABLE siret ADD COLUMN adresse_complete;')

In [56]:
# Insert addresses into column
start_time = time.time()
cursor.execute(f'''UPDATE siret
                    SET adresse_complete = (
                                            SELECT add_adresse_complete(COALESCE(complement_adresse,''),COALESCE(numero_voie,''),
                                                                    COALESCE(indice_repetition,''), COALESCE(type_voie,''),
                                                                    COALESCE(libelle_voie,''), COALESCE(libelle_commune,''),
                                                                    COALESCE(libelle_cedex,''), COALESCE(distribution_speciale,''),
                                                                    COALESCE(commune,''), COALESCE(cedex,''), COALESCE(libelle_commune_etranger,''),
                                                                    COALESCE(libelle_pays_etranger,'')))''')
stats()

--- 176.37868094444275 seconds ---
used memory : 14.6Go


In [57]:
for row in cursor.execute('SELECT adresse_complete FROM siret WHERE siren="005520135" LIMIT 10;'):
    print(row)

('70 RUE DE LAUSANNE 01143 DIVONNE-LES-BAINS',)
('32 PL DE LA GARE 59350 LILLE',)
('253 RUE DE MENCHECOURT 80001 ABBEVILLE',)


In [58]:
# Create adresses table with grouped addresses for each siren
cursor.execute(f'''DROP TABLE IF EXISTS adresses''')
cursor.execute('''CREATE TABLE adresses (siren VARCHAR(10), liste_adresses)''')
cursor.execute('''
                CREATE UNIQUE INDEX index_liste_adresses
                ON adresses(siren);
                ''')
connection.commit()

In [59]:
start_time = time.time()
# Insert addresses into adresses table
cursor.execute('''INSERT INTO adresses (siren, liste_adresses) SELECT siren, GROUP_CONCAT(adresse_complete, ',') as liste_adresses FROM siret GROUP BY siren;''')
stats()

--- 53.64046859741211 seconds ---
used memory : 14.7Go


In [60]:
for row in cursor.execute('SELECT * FROM adresses WHERE siren="005520135" LIMIT 10;'):
    print(row)

('005520135', '70 RUE DE LAUSANNE 01143 DIVONNE-LES-BAINS,32 PL DE LA GARE 59350 LILLE,253 RUE DE MENCHECOURT 80001 ABBEVILLE')


In [61]:
cursor.execute('SELECT * FROM adresses LIMIT 1')
names = [description[0] for description in cursor.description]
rows = cursor.fetchall()
for row in rows:
    for name,val in zip(names,row):
        print(f"{name}: {val}")

siren: 000325175
liste_adresses: 170 AV DU COL DE L ANGE 13042 GEMENOS,61 RUE MARX DORMOY 13204 MARSEILLE 4,51 RUE MARX DORMOY 13204 MARSEILLE 4,1 PL LEONCE DE SEYNES 84007 AVIGNON,ECONOMIS 6 AV FRANCOIS MAURIAC 84007 AVIGNON


## Siège only

In [62]:
cursor.execute(f'''DROP TABLE IF EXISTS siretsiege''')
cursor.execute(f'''CREATE TABLE IF NOT EXISTS siretsiege
        (
        id INTEGER NOT NULL PRIMARY KEY,
        siren,
        siret,
        date_creation,
        tranche_effectif_salarie,
        activite_principale_registre_metier,
        is_siege,
        numero_voie,
        type_voie,
        libelle_voie,
        code_postal,
        libelle_cedex,
        libelle_commune,
        commune,
        complement_adresse,
        complement_adresse_2,
        numero_voie_2,
        indice_repetition_2,
        type_voie_2,
        libelle_voie_2,
        commune_2,
        libelle_commune_2,
        cedex_2,
        libelle_cedex_2,
        cedex,
        date_debut_activite,
        distribution_speciale,
        distribution_speciale_2,
        etat_administratif_etablissement,
        enseigne_1,
        enseigne_2,
        enseigne_3,
        activite_principale,
        indice_repetition,
        nom_commercial,
        libelle_commune_etranger,
        code_pays_etranger,
        libelle_pays_etranger,
        libelle_commune_etranger_2,
        code_pays_etranger_2,
        libelle_pays_etranger_2,
        longitude,
        latitude,
        geo_adresse,
        geo_id,
        adresse_complete)
''')
cursor.execute('''INSERT INTO siretsiege (
        siren,
        siret,
        date_creation,
        tranche_effectif_salarie,
        activite_principale_registre_metier,
        is_siege,
        numero_voie,
        type_voie,
        libelle_voie,
        code_postal,
        libelle_cedex,
        libelle_commune,
        commune,
        complement_adresse,
        complement_adresse_2,
        numero_voie_2,
        indice_repetition_2,
        type_voie_2,
        libelle_voie_2,
        commune_2,
        libelle_commune_2,
        cedex_2,
        libelle_cedex_2,
        cedex,
        date_debut_activite,
        distribution_speciale,
        distribution_speciale_2,
        etat_administratif_etablissement,
        enseigne_1,
        enseigne_2,
        enseigne_3,
        activite_principale,
        indice_repetition,
        nom_commercial,
        libelle_commune_etranger,
        code_pays_etranger,
        libelle_pays_etranger,
        libelle_commune_etranger_2,
        code_pays_etranger_2,
        libelle_pays_etranger_2,
        longitude,
        latitude,
        geo_adresse,
        geo_id,
        adresse_complete) 
    SELECT
        siren,
        siret,
        date_creation,
        tranche_effectif_salarie,
        activite_principale_registre_metier,
        is_siege,
        numero_voie,
        type_voie,
        libelle_voie,
        code_postal,
        libelle_cedex,
        libelle_commune,
        commune,
        complement_adresse,
        complement_adresse_2,
        numero_voie_2,
        indice_repetition_2,
        type_voie_2,
        libelle_voie_2,
        commune_2,
        libelle_commune_2,
        cedex_2,
        libelle_cedex_2,
        cedex,
        date_debut_activite,
        distribution_speciale,
        distribution_speciale_2,
        etat_administratif_etablissement,
        enseigne_1,
        enseigne_2,
        enseigne_3,
        activite_principale,
        indice_repetition,
        nom_commercial,
        libelle_commune_etranger,
        code_pays_etranger,
        libelle_pays_etranger,
        libelle_commune_etranger_2,
        code_pays_etranger_2,
        libelle_pays_etranger_2,
        longitude,
        latitude,
        geo_adresse,
        geo_id,
        adresse_complete
    FROM siret
    WHERE is_siege = 'true';
''')

In [63]:
cursor.execute('SELECT * FROM siretsiege WHERE siren="965706690" LIMIT 1')
names = [description[0] for description in cursor.description]
rows = cursor.fetchall()
for row in rows:
    for name,val in zip(names,row):
        print(f"{name}: {val}")

id: 1162509
siren: 965706690
siret: 96570669000017
date_creation: None
tranche_effectif_salarie: None
activite_principale_registre_metier: None
is_siege: true
numero_voie: None
type_voie: AV
libelle_voie: SAINT SYLVESTRE
code_postal: 06100
libelle_cedex: None
libelle_commune: NICE
commune: 06088
complement_adresse: None
complement_adresse_2: None
numero_voie_2: None
indice_repetition_2: None
type_voie_2: None
libelle_voie_2: None
commune_2: None
libelle_commune_2: None
cedex_2: None
libelle_cedex_2: None
cedex: None
date_debut_activite: 1984-12-25
distribution_speciale: None
distribution_speciale_2: None
etat_administratif_etablissement: F
enseigne_1: None
enseigne_2: None
enseigne_3: None
activite_principale: 00.97
indice_repetition: None
nom_commercial: None
libelle_commune_etranger: None
code_pays_etranger: None
libelle_pays_etranger: None
libelle_commune_etranger_2: None
code_pays_etranger_2: None
libelle_pays_etranger_2: None
longitude: 7.24991
latitude: 43.725432
geo_adresse: Ave

In [64]:
cursor.execute('''
                CREATE INDEX index_siret_siren
                ON siretsiege (siren);
                ''')

### Check tables

In [65]:
cursor.execute("select * from SQLite_master")

tables = cursor.fetchall()
print("Listing tables and indices from main database:")

for table in tables:

    print("Type of database object: %s"%(table[0]))

    print("Name of the database object: %s"%(table[1]))

    print("Table Name: %s"%(table[2]))

    print("Root page: %s"%(table[3]))

    print("SQL statement: %s"%(table[4]))

Listing tables and indices from main database:
Type of database object: table
Name of the database object: unite_legale
Table Name: unite_legale
Root page: 2
SQL statement: CREATE TABLE unite_legale
    (
        siren,
        date_creation_unite_legale,
        sigle,
        prenom,
        identifiant_association_unite_legale,
        tranche_effectif_salarie_unite_legale,
        date_mise_a_jour_unite_legale,
        categorie_entreprise,
        etat_administratif_unite_legale,
        nom,
        nom_usage,
        nom_raison_sociale,
        nature_juridique_unite_legale,
        activite_principale_unite_legale,
        economie_sociale_solidaire_unite_legale
    )
Type of database object: index
Name of the database object: index_siren
Table Name: unite_legale
Root page: 3
SQL statement: CREATE UNIQUE INDEX index_siren
                ON unite_legale (siren)
Type of database object: table
Name of the database object: siret
Table Name: siret
Root page: 616488
SQL statement: C

# Elasticsearch

In [66]:
elastic_url = 'http://search.sirene.dataeng.etalab.studio:80/'
elastic_index = 'siren-'
elastic_user = 'elastic'
elastic_password = 'etalab123'
elastic_index_shards = 1
elastic_bulk_size=1500

## Connection

In [67]:
# initiate the default connection to elasticsearch
connections.create_connection(
    hosts=[elastic_url],
    http_auth=(elastic_user, elastic_password),
    retry_on_timeout=True,
)

<Elasticsearch([{'host': 'search.sirene.dataeng.etalab.studio', 'port': 80}])>

## Mapping

In [68]:
from elasticsearch_dsl import (
    Boolean,
    Date,
    Document,
    GeoPoint,
    Integer,
    Keyword,
    Text,
    analyzer,
    token_filter,
    tokenizer,
)

# Define filters
french_elision = token_filter(
    "french_elision",
    type="elision",
    articles_case=True,
    articles=[
        "l",
        "m",
        "t",
        "qu",
        "n",
        "s",
        "j",
        "d",
        "c",
        "jusqu",
        "quoiqu",
        "lorsqu",
        "puisqu",
    ],
)
french_stop = token_filter("french_stop", type="stop", stopwords="_french_")
french_stemmer = token_filter("french_stemmer", type="stemmer", language="light_french")
# ignore_case option deprecated, use lowercase filter before synonym filter
french_synonym = token_filter(
    "french_synonym", type="synonym", expand=True, synonyms=[]
)

# Define analyzer
annuaire_analyzer = analyzer(
    "annuaire_analyzer",
    tokenizer=tokenizer("icu_tokenizer"),
    filter=[
        "lowercase",
        french_elision,
        french_stop,
        "icu_folding",
        french_synonym,
        "asciifolding",
        french_stemmer,
    ],
)


class Siren(Document):
    """
    Class used to represent a company headquarters,
    one siren number and the corresponding sheadquarters siret number
    """

    activite_principale_siege = Keyword()  # Add index_prefixes option
    activite_principale_unite_legale = Keyword()
    activite_principale_registre_metier = Keyword()
    adresse_etablissement = Text()
    adresse_etablissement_2 = Text()
    categorie_entreprise = Text()
    cedex = Keyword()
    code_pays_etranger = Text()
    code_postal = Keyword()
    commune = Keyword()
    complement_adresse = Text()
    concat_enseigne_adresse = Text(analyzer=annuaire_analyzer)
    concat_nom_adr_siren = Text(
        analyzer=annuaire_analyzer, fields={"keyword": Keyword()}
    )
    coordonnees = GeoPoint()
    date_creation_siege = Date()
    date_creation_unite_legale = Date()
    date_debut_activite_siege = Date()
    date_mise_a_jour = Date()
    departement = Keyword()
    distribution_speciale = Text()
    economie_sociale_solidaire_unite_legale = Keyword()
    enseigne = Text()
    etat_administratif_unite_legale = Keyword()
    etat_administratif_siege = Keyword()
    geo_adresse = Text(analyzer=annuaire_analyzer)
    geo_id = Keyword()
    identifiant_association_unite_legale = Keyword()
    indice_repetition = Text()
    is_entrepreneur_individuel = Boolean()
    is_siege = Boolean()
    latitude = Text()
    libelle_cedex = Text()
    libelle_commune = Text()
    libelle_commune_etranger = Text()
    libelle_pays_etranger = Text()
    libelle_voie = Text()
    liste_adresse = Text(analyzer=annuaire_analyzer)
    liste_enseigne = Text(analyzer=annuaire_analyzer)
    longitude = Text()
    nature_juridique_unite_legale = Integer()
    nom = Text()
    nom_complet = Text(analyzer=annuaire_analyzer, fields={"keyword": Keyword()})
    nom_raison_sociale = Text()
    nombre_etablissements = Integer()  # NaN can't be stored in an integer array
    nombre_etablissements_ouverts = Integer()
    numero_voie = Text()
    prenom = Keyword()
    section_activite_principale = Keyword()
    sigle = Keyword()
    siren = Keyword(required=True)
    siret_siege = Keyword(required=True)
    type_voie = Text()
    tranche_effectif_salarie_siege = Keyword()
    tranche_effectif_salarie_unite_legale = Keyword()

    class Index:
        name = elastic_index
        settings = {"number_of_shards": elastic_index_shards, "number_of_replicas": 0}

## Create Index

In [69]:
elastic_connection = connections.get_connection()
elastic_mapping = Siren._index.get_mapping()
elastic_health = elastic_connection.cluster.health()
elastic_status = elastic_health["status"]

In [70]:
if Index(elastic_index).exists():
    logging.info(f"Index  {elastic_index} already exists! Deleting...")
    Index(elastic_index).delete()
    logging.info(f"Index {elastic_index} deleted!")
logging.info(f"Creating {elastic_index} index!")
Siren.init()

INFO:root:Index  siren- already exists! Deleting...
INFO:elasticsearch:DELETE http://search.sirene.dataeng.etalab.studio:80/siren- [status:200 request:0.956s]
INFO:root:Index siren- deleted!
INFO:root:Creating siren- index!
INFO:elasticsearch:PUT http://search.sirene.dataeng.etalab.studio:80/siren- [status:200 request:0.037s]


## Health check

In [71]:
if elastic_status not in ("green", "yellow"):
    raise Exception(
        f"Cluster status is {self.elastic_status}, not green nor yellow!!"
    )
else:
    logging.info(f"Cluster status is functional: {elastic_status}")

INFO:root:Cluster status is functional: green


## Indexing

In [72]:
def dict_from_row(row):
    return dict(zip(row.keys(), row))

In [73]:
def process_res(res):
    arr = []
    for result in res:            
        mydict = {}
        # mydict['siege'] = {}
        for item in result:
            # clean string from spaces
            if item == "liste_enseignes":
                mydict[item] = unique_string(result[item])
            else:   
                mydict[item] = result[item]

        mydict['nom_complet'] = create_nom_complet(
            result['nature_juridique_unite_legale'],
            result['nom'],
            result['nom_usage'],
            result['nom_raison_sociale'],
            result['sigle'],
            result['prenom'],
        )    
        mydict['is_entrepreneur_individuel'] = create_entrepreneur_individuel(
            result['nature_juridique_unite_legale']
        )
        mydict['section_activite_principale'] = create_section(
            result['activite_principale_unite_legale']
        )
        mydict['departement'] = create_departement(
            result['commune']
        )
        mydict['coordonnees'] = create_coordonnees(
            result['longitude'],
            result['latitude']
        )
        mydict['concat_enseigne_adresse'] = (get_string(unique_string(result['liste_enseignes'])) + " " + get_string(unique_string(result['liste_adresses']))).strip()
        mydict['concat_nom_adr_siren'] = (get_string(mydict['nom_complet']) + " " + get_string(result['adresse_complete']) + " " + get_string(result['siren'])).strip()
        
        arr.append(mydict)
    return arr
    

In [74]:
def doc_generator(data):
    for index, document in enumerate(data):
        yield Siren(meta={"id": document["siret_siege"]}, **document).to_dict(
            include_meta=True
        )
    # Serialize the instance into a dictionary so that it can be saved in elasticsearch.

In [75]:
global_stats()

--- 1471.633181810379 seconds ---
used memory : 14.7Go


In [76]:
chunk_size = 1500

In [77]:
for row in cursor.execute('''SELECT count(*) FROM siretsiege;'''):
    nb_iter = int(row[0]) / chunk_size + 1

In [78]:
int(nb_iter)

15456

In [79]:
start_time = time.time()

cursor.execute(f'''
    SELECT 
        ul.siren,
        st.siret as siret_siege,
        st.date_creation as date_creation_siege,
        st.tranche_effectif_salarie as tranche_effectif_salarie_siege,
        st.date_debut_activite as date_debut_activite_siege,
        st.etat_administratif_etablissement as etat_administratif_siege,
        st.activite_principale as activite_principale_siege,
        st.complement_adresse as complement_adresse,
        st.numero_voie as numero_voie,
        st.indice_repetition as ndice_repetition,
        st.type_voie as type_voie,
        st.libelle_voie as libelle_voie,
        st.distribution_speciale as distribution_speciale,
        st.cedex as cedex,
        st.libelle_cedex as libelle_cedex,
        st.commune as commune,
        st.libelle_commune as libelle_commune,
        st.code_pays_etranger as code_pays_etranger,
        st.libelle_commune_etranger as libelle_commune_etranger,
        st.libelle_pays_etranger as libelle_pays_etranger,
        st.code_postal as code_postal,
        st.geo_id as geo_id,
        st.longitude as longitude,
        st.latitude as latitude,
        st.activite_principale_registre_metier as activite_principale_registre_metier,
        st.adresse_complete as adresse_complete,
        ul.date_creation_unite_legale as date_creation_unite_legale,
        ul.tranche_effectif_salarie_unite_legale as tranche_effectif_salarie_unite_legale,
        ul.date_mise_a_jour_unite_legale as date_mise_a_jour,
        ul.categorie_entreprise as categorie_entreprise,
        ul.etat_administratif_unite_legale as etat_administratif_unite_legale,
        ul.nom_raison_sociale as nom_raison_sociale,
        ul.nature_juridique_unite_legale as nature_juridique_unite_legale,
        ul.activite_principale_unite_legale as activite_principale_unite_legale,
        ul.economie_sociale_solidaire_unite_legale as economie_sociale_solidair_unite_legale,
        (SELECT count FROM count_etab ce WHERE ce.siren = st.siren) as nombre_etablissements,
        (SELECT count FROM count_etab_ouvert ceo WHERE ceo.siren = st.siren) as nombre_etablissements_ouverts,
        (SELECT liste_enseignes FROM enseignes le WHERE le.siren = st.siren) as liste_enseignes,
        (SELECT liste_adresses FROM adresses la WHERE la.siren = st.siren) as liste_adresses,
        ul.sigle as sigle,
        ul.prenom as prenom,
        ul.nom as nom,
        ul.nom_usage as nom_usage,
        st.is_siege as is_siege
    FROM
        siretsiege st
    LEFT JOIN unite_legale ul 
    ON
        ul.siren = st.siren        
''')

In [80]:
i=0    
res = 1

while (res):
    res = cursor.fetchmany(1500)
    columns = tuple([x[0] for x in cursor.description])
    res = tuple(
        [{column: val for column, val in zip(columns, x)} for x in res]
    )
    res2 = process_res(res)
    i=i+1
    if(i%1000 == 0):
        print(i)
    try:
        for success, details in helpers.parallel_bulk(elastic_connection, doc_generator(res2), chunk_size=elastic_bulk_size):
            if not success:
                raise Exception(f"A file_access document failed: {details}")
    except Exception as e:
        logging.error(f"Failed to send to Elasticsearch: {e}")
    doc_count = elastic_connection.cat.count(
            index=elastic_index, params={"format": "json"}
        )[0]["count"]
    print(f"Number of documents indexed: {doc_count}")
stats()

INFO:elasticsearch:POST http://search.sirene.dataeng.etalab.studio:80/_bulk [status:200 request:0.257s]
INFO:elasticsearch:GET http://search.sirene.dataeng.etalab.studio:80/_cat/count/siren-?format=json [status:200 request:0.002s]


Number of documents indexed: 0


INFO:elasticsearch:POST http://search.sirene.dataeng.etalab.studio:80/_bulk [status:200 request:0.118s]
INFO:elasticsearch:GET http://search.sirene.dataeng.etalab.studio:80/_cat/count/siren-?format=json [status:200 request:0.002s]


Number of documents indexed: 1500


INFO:elasticsearch:POST http://search.sirene.dataeng.etalab.studio:80/_bulk [status:200 request:0.117s]
INFO:elasticsearch:GET http://search.sirene.dataeng.etalab.studio:80/_cat/count/siren-?format=json [status:200 request:0.002s]


...

INFO:elasticsearch:POST http://search.sirene.dataeng.etalab.studio:80/_bulk [status:200 request:0.166s]
INFO:elasticsearch:GET http://search.sirene.dataeng.etalab.studio:80/_cat/count/siren-?format=json [status:200 request:0.002s]


Number of documents indexed: 19797000


INFO:elasticsearch:POST http://search.sirene.dataeng.etalab.studio:80/_bulk [status:200 request:0.178s]
INFO:elasticsearch:GET http://search.sirene.dataeng.etalab.studio:80/_cat/count/siren-?format=json [status:200 request:0.002s]


Number of documents indexed: 19797000


INFO:elasticsearch:POST http://search.sirene.dataeng.etalab.studio:80/_bulk [status:200 request:0.195s]
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [81]:
global_stats()

--- 10931.187263011932 seconds ---
used memory : 14.7Go
